In [1]:
import pandas as pd
from lifelines import CoxPHFitter

In [2]:
import pickle
with open('data/learned_cox_model.pkl','rb') as f:
  cf = pickle.load(f)

In [3]:
data = pd.read_csv("data/merged_dataset_python.csv", index_col=0)

In [4]:
data["is_maried"] = data["is_maried"].astype(int)

In [5]:
# Exclude all fields except mentioned here
usefields = ["is_maried", "age", "work_experience", "children_number", "has_mortgage",
            "loan_products_nii", "checking_account_balance", "deposit_balance", "money_market_balance",
            "investable_asset_indicator", "Sentiment", "Sentiment_days_passed", 
            "Day_Observed", "Event"]
data = data[usefields]

In [6]:
predict = cf.predict_survival_function(data)

In [7]:
res = pd.DataFrame()
for i in [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 365]:
    row = predict[:i].tail(1)
    row.index = [i]
    res = pd.concat([res, row])

In [8]:
# Define Final churn rate as 
Final_churn = ((1 - res.loc[365]) > 0.4).astype(int)
Final_churn.name = "Predicted churn"
Actual_churn = data.loc[:,"Event"]
Actual_churn.name = "Actual churn"
res = res.append([Final_churn, Actual_churn])

In [9]:
print "Accuracy = {0}%".format(100.0*sum(Actual_churn == Final_churn)/len(Final_churn))

Accuracy = 89.5102%


In [10]:
res.T.to_csv("data/churn_prediction.csv")